# Hyperparameter Tuning Demo using Keras Tuner

by Cedric Conol

Notebook from the article: [Hyperparameter Tuning with Keras Tuner](https://towardsdatascience.com/hyperparameter-tuning-with-keras-tuner-283474fbfbe)

### Load the data

In [1]:
from tensorflow.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

57344/57026 [==============================] - 0s 1us/step


### Check tensorflow and kerastuner versions

In [3]:
import tensorflow as tf
import kerastuner as kt

print(tf.__version__)
print(kt.__version__)

2.4.1
1.0.2


### Import libraries

In [17]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import models, layers
from kerastuner import HyperModel, RandomSearch, Hyperband, BayesianOptimization
from keras.models import Sequential # Own addition
import os # Own addition

### Set random seeed

In [5]:
from numpy.random import seed
seed(42)
import tensorflow
tensorflow.random.set_seed(42)

### Model without Hyperparameter Tuning

In [6]:
scaler = StandardScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

model = models.Sequential()
model.add(layers.Dense(8, activation='relu', input_shape=(x_train.shape[1],),
                       kernel_initializer='zeros', bias_initializer='zeros'))
model.add(layers.Dense(16, activation='relu', kernel_initializer='zeros',
                       bias_initializer='zeros'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1))

model.compile(optimizer='rmsprop',loss='mse',metrics=['mse'])

In [7]:
history = model.fit(x_train_scaled, y_train, validation_split=0.2, epochs=10)

Epoch 1/10
11/11 [==============================] - 3s 183ms/step - loss: 552.3023 - mse: 552.3023 - val_loss: 642.7367 - val_mse: 642.7367
Epoch 2/10
11/11 [==============================] - 0s 13ms/step - loss: 551.8933 - mse: 551.8933 - val_loss: 642.1722 - val_mse: 642.1722
Epoch 3/10
11/11 [==============================] - 0s 10ms/step - loss: 586.9997 - mse: 586.9997 - val_loss: 641.6613 - val_mse: 641.6613
Epoch 4/10
11/11 [==============================] - 0s 7ms/step - loss: 580.1702 - mse: 580.1702 - val_loss: 641.1359 - val_mse: 641.1359
Epoch 5/10
11/11 [==============================] - 0s 9ms/step - loss: 589.8893 - mse: 589.8893 - val_loss: 640.6212 - val_mse: 640.6212
Epoch 6/10
11/11 [==============================] - 0s 8ms/step - loss: 578.4940 - mse: 578.4940 - val_loss: 640.1004 - val_mse: 640.1004
Epoch 7/10
11/11 [==============================] - 0s 10ms/step - loss: 577.4522 - mse: 577.4522 - val_loss: 639.5798 - val_mse: 639.5798
Epoch 8/10
11/11 [===========

In [8]:
mse = model.evaluate(x_test_scaled, y_test)[1]

4/4 [==============================] - 0s 2ms/step - loss: 610.3947 - mse: 610.3947


In [9]:
print('MSE without tuning: {}'.format(mse))

MSE without tuning: 610.3947143554688


### Building the Hypermodel

In [10]:
class RegressionHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        model = Sequential()
        model.add(
            layers.Dense(
                units=hp.Int('units', 8, 64, 4, default=8),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                input_shape=input_shape,
                kernel_initializer='zeros', bias_initializer='zeros'
            )
        )
        
        model.add(
            layers.Dense(
                units=hp.Int('units', 16, 64, 4, default=16),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                kernel_initializer='zeros', bias_initializer='zeros'
            )
        )
        
        model.add(
            layers.Dropout(
                hp.Float(
                    'dropout',
                    min_value=0.0,
                    max_value=0.1,
                    default=0.005,
                    step=0.01)
            )
        )
        
        model.add(layers.Dense(1, kernel_initializer='zeros', bias_initializer='zeros'))
        
        model.compile(
            optimizer='rmsprop',loss='mse',metrics=['mse']
        )
        
        return model

### Initialize hypermodel

In [11]:
input_shape = (x_train.shape[1],)
hypermodel = RegressionHyperModel(input_shape)

### Build Random Search Tuner

In [18]:
tuner_rs = RandomSearch(
            hypermodel,
            objective='mse',
            seed=42,
            max_trials=10,
            executions_per_trial=2, overwrite=True,
            directory=os.path.normpath('D:/ML_Keras_Tuner/Tests/RandomSearch_test') # Own addition, removes error
        )

### Run Random Search

In [19]:
tuner_rs.search(x_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=0)

INFO:tensorflow:Oracle triggered exit


### Evaluate Random Search

In [20]:
best_model = tuner_rs.get_best_models(num_models=1)[0]
mse_rs = best_model.evaluate(x_test_scaled, y_test)[1]

4/4 [==============================] - 0s 1ms/step - loss: 533.3783 - mse: 533.3783


In [21]:
print('Random search MSE: ', mse_rs)

Random search MSE:  561.71337890625


### Build, Run and Evaluate Hyperband Tuner

In [24]:
tuner_hb = Hyperband(
            hypermodel,
            max_epochs=5,
            objective='mse',
            seed=42,
            executions_per_trial=2,
            directory=os.path.normpath('D:/ML_Keras_Tuner/Tests/Hyperband_test') # Own addition, removes error
        )

tuner_hb.search(x_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=0)

best_model = tuner_hb.get_best_models(num_models=1)[0]
mse_hb = best_model.evaluate(x_test_scaled, y_test)[1]

INFO:tensorflow:Oracle triggered exit
4/4 [==============================] - 0s 2ms/step - loss: 560.8518 - mse: 560.8518


In [25]:
print('Hyperband MSE: ', mse_hb)

Hyperband MSE:  589.93017578125


### Build, Run and Evaluate Bayesian Optimization Tuner

In [26]:
tuner_bo = BayesianOptimization(
            hypermodel,
            objective='mse',
            max_trials=10,
            seed=42,
            executions_per_trial=2,
            directory=os.path.normpath('D:/ML_Keras_Tuner/Tests/BayesianOptimization_test') # Own addition, removes error
        )

tuner_bo.search(x_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=0)

best_model = tuner_bo.get_best_models(num_models=1)[0]
mse_bo = best_model.evaluate(x_test_scaled, y_test)[1]

C:\Users\17832020\Anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:201: RuntimeWarning: invalid value encountered in true_divide
  y = (y - self._y_train_mean) / self._y_train_std
C:\Users\17832020\Anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:201: RuntimeWarning: invalid value encountered in true_divide
  y = (y - self._y_train_mean) / self._y_train_std
C:\Users\17832020\Anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:201: RuntimeWarning: invalid value encountered in true_divide
  y = (y - self._y_train_mean) / self._y_train_std
C:\Users\17832020\Anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:201: RuntimeWarning: invalid value encountered in true_divide
  y = (y - self._y_train_mean) / self._y_train_std


INFO:tensorflow:Oracle triggered exit
4/4 [==============================] - 0s 2ms/step - loss: 433.0382 - mse: 433.0382


In [27]:
print('Bayesian Optimization MSE: ', mse_bo)

Bayesian Optimization MSE:  458.7108154296875
